<a href="https://colab.research.google.com/github/lmassaron/ml4dummies_3ed/blob/main/ML4D3E_17_scoring_opinions_and_sentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
text_1 = "The quick brown fox jumps over the lazy dog."
text_2 = "My dog is quick and can jump over fences."
text_3 = \
      "Your dog is so lazy that it sleeps all the day."
corpus = [text_1, text_2, text_3]

In [3]:
from sklearn.feature_extraction.text import (
    CountVectorizer)

vectorizer = CountVectorizer(binary=True,
                             lowercase=False)
vectorizer.fit(corpus)
vectorized_text = vectorizer.transform(corpus)
print(vectorized_text.todense())

[[0 1 0 0 0 1 0 0 1 0 1 0 0 0 1 1 1 1 0 0 0 1]
 [1 0 0 0 1 0 1 0 1 1 0 1 0 1 0 0 1 1 0 0 0 0]
 [0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 1 1]]


In [4]:
print(vectorizer.vocabulary_)

{'The': 1, 'quick': 17, 'brown': 5, 'fox': 10, 'jumps': 14, 'over': 16, 'the': 21, 'lazy': 15, 'dog': 8, 'My': 0, 'is': 11, 'and': 4, 'can': 6, 'jump': 13, 'fences': 9, 'Your': 2, 'so': 19, 'that': 20, 'it': 12, 'sleeps': 18, 'all': 3, 'day': 7}


In [5]:
text_4 = \
    "A black dog just passed by but my dog is brown."
corpus.append(text_4)
vectorizer = CountVectorizer()
vectorizer.fit(corpus)
vectorized_text = vectorizer.transform(corpus)
print(vectorized_text.todense()[-1])

[[0 0 1 1 1 1 0 0 2 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0]]


In [6]:
from sklearn.feature_extraction.text import (
    TfidfTransformer)

tfidf = TfidfTransformer(norm="l1")
tfidf_mtx = tfidf.fit_transform(vectorized_text)

phrase = 3 # choose a number from 0 to 3

total = 0
for word in vectorizer.vocabulary_:
    pos = vectorizer.vocabulary_[word]
    value = list(tfidf_mtx.toarray()[phrase])[pos]
    if value !=0.0:
        print(f"{word:7s}: {value:0.3f}")
        total += value
print(f"\nSummed values of a phrase: {total:.1f}")

brown  : 0.095
dog    : 0.126
my     : 0.095
is     : 0.077
black  : 0.121
just   : 0.121
passed : 0.121
by     : 0.121
but    : 0.121

Summed values of a phrase: 1.0


In [7]:
bigrams = CountVectorizer(ngram_range=(2, 2))
print(bigrams.fit(corpus).vocabulary_)

{'the quick': 30, 'quick brown': 24, 'brown fox': 3, 'fox jumps': 9, 'jumps over': 15, 'over the': 21, 'the lazy': 29, 'lazy dog': 17, 'my dog': 19, 'dog is': 7, 'is quick': 11, 'quick and': 23, 'and can': 1, 'can jump': 6, 'jump over': 14, 'over fences': 20, 'your dog': 31, 'is so': 12, 'so lazy': 26, 'lazy that': 18, 'that it': 27, 'it sleeps': 13, 'sleeps all': 25, 'all the': 0, 'the day': 28, 'black dog': 2, 'dog just': 8, 'just passed': 16, 'passed by': 22, 'by but': 5, 'but my': 4, 'is brown': 10}


In [8]:
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
from sklearn.feature_extraction.text import (
    CountVectorizer)

from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

stemmer = PorterStemmer()
stop_words = stopwords.words("english")

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens
              if token not in stop_words]
    stems = stem_tokens(tokens, stemmer)
    return stems

docs = ["Sam loves swimming so he swims all the time"]
vect = CountVectorizer(tokenizer=tokenize)
vec = vect.fit(docs)

sentence1 = vec.transform(
    ["George loves swimming too! "])

print(vec.get_feature_names_out())
print(sentence1.toarray())

['love' 'sam' 'swim' 'time']
[[1 0 1 0]]


In [10]:
import pandas as pd

repository = \
    "https://github.com/lmassaron/ml4dummies_3ed/"
release = "releases/download/v1.0/"
filename = repository + release + "imdb_50k.csv"
reviews = pd.read_csv(filename)

In [11]:
reviews.sentiment.value_counts()

,count
sentiment,
1,25000
0,25000


In [12]:
print(reviews.review.sample(1).values[0])

Hey guy, this movies is everything about choices. All the times in your life you must pick something or it just pass away... And this movie prove that! Of course, life in fact is not like a beautiful picture as this movie shows... it not shows indeed but some may figure that. I'm trying to say it's full of pain, love and deep lessons of live. Aaron, the Mormon missionary is the real shepherd digging out the thing beautiful deep inside Chisthian, the skin feeling guy...<br /><br />It's a great end and you do always believe in fate because it will surprise you in a turn or in other of your live like Latter Days...<br /><br />Big deal watch it!!!


In [13]:
from sklearn.model_selection import train_test_split

train, temp = train_test_split(
    reviews, test_size=0.4, random_state=0)
valid, test = train_test_split(
    temp, test_size=0.5, random_state=0)

print(f"Train size: {len(train)}")
print(f"Validation size: {len(valid)}")
print(f"Test size: {len(test)}")

Train size: 30000
Validation size: 10000
Test size: 10000


In [14]:
import os
os.environ["KERAS_BACKEND"] = "jax"
os.environ["HF_HUB_DISABLE_IMPLICIT_TOKEN"] = "1"

In [15]:
import keras

maxlen = 256
vocab_size_limit = 10000

text_vectorization = keras.layers.TextVectorization(
    max_tokens=vocab_size_limit,
    output_mode='int',
    output_sequence_length=maxlen,
    pad_to_max_tokens=True)

text_vectorization.adapt(train.review.values)

def vectorize_text_data(df, vectorizer):
    sequences = vectorizer(df.review.values)
    return sequences, df.sentiment.values

X, y = vectorize_text_data(train, text_vectorization)
Xv, yv = vectorize_text_data(valid, text_vectorization)
Xt, yt = vectorize_text_data(test, text_vectorization)

In [16]:
keras.utils.set_random_seed(0)

model = keras.models.Sequential()
vocab_size = text_vectorization.vocabulary_size()
embedding_dim = 64

model.add(keras.layers.Input(shape=(maxlen,)))
model.add(keras.layers.Embedding(input_dim=vocab_size,
                              output_dim=embedding_dim))
model.add(keras.layers.Bidirectional(
    keras.layers.LSTM(32, return_sequences=True)))
model.add(keras.layers.Bidirectional(
    keras.layers.LSTM(32, return_sequences=False)))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 256, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 256, 64)        │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 64)             │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 689,729 (2.63 MB)

 Trainable params: 689,729 (2.63 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
history = model.fit(X, y, epochs=2, batch_size=8,
                    validation_data=(Xv, yv))

Epoch 1/2
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 136s 35ms/step - accuracy: 0.6468 - loss: 0.6230 - val_accuracy: 0.8466 - val_loss: 0.3587
Epoch 2/2
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 130s 34ms/step - accuracy: 0.8769 - loss: 0.3110 - val_accuracy: 0.8802 - val_loss: 0.2800


In [18]:
from sklearn.metrics import accuracy_score

predictions = (model.predict(Xt)>=0.5).astype(int)
test_accuracy = accuracy_score(yt, predictions)
print(f"Accuracy on test set: {test_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step
Accuracy on test set: 0.8828


In [19]:
from datasets import Dataset
from transformers import AutoTokenizer

model_name = "answerdotai/ModernBERT-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"],
                     padding="max_length",
                     truncation=True,
                     max_length=256)

def tokenize_dataset(data):
  data_dict = {'text': data['review'].values,
               'labels': data['sentiment'].values}
  dataset = Dataset.from_dict(data_dict)
  return dataset.map(tokenize_function, batched=True)

tokenized_train_dataset = tokenize_dataset(train)
tokenized_valid_dataset = tokenize_dataset(valid)
tokenized_test_dataset = tokenize_dataset(test)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [20]:
from transformers import (
    AutoModelForSequenceClassification as AutoModel)

model = AutoModel.from_pretrained(model_name,
                                  num_labels=2)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to="none",
    eval_strategy="steps")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset)

train_result = trainer.train()

W0817 18:08:02.287000 1100 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


Step,Training Loss,Validation Loss
500,0.339900,0.193214
1000,0.263900,0.195235
1500,0.237900,0.206862
2000,0.230000,0.207635
2500,0.228900,0.208621
3000,0.204100,0.198057
3500,0.186400,0.196846


In [22]:
import numpy as np
from sklearn.metrics import accuracy_score

predictions = trainer.predict(tokenized_test_dataset)
predicted_labels = np.argmax(predictions.predictions,
                             axis=1)
test_accuracy = accuracy_score(test['sentiment'].values,
                               predicted_labels)
print(f"Accuracy on test set: {test_accuracy}")

Accuracy on test set: 0.9427
